#실행전 준비

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

In [ ]:
!pip install -q -U transformers bitsandbytes protobuf
!pip install -q -U peft gradio langchain_chroma langchain_huggingface langchain_community

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files

src = list(files.upload().values())[0]
open('summerizer.py','wb').write(src)

Saving summerizer_1.py to summerizer_1.py


2930

In [6]:
from google.colab import files

src = list(files.upload().values())[0]
open('predictor.py','wb').write(src)

Saving predictor_1.py to predictor_1 (2).py


3581

In [2]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

#라이브러리 import

In [7]:
import pickle
import gradio as gr
from summerizer import summarize_text
from predictor import predict_law

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#test 데이터 로드

In [10]:
with open("./final_test.pkl","rb") as fr:
    final_test = pickle.load(fr)

In [11]:
type(final_test)

datasets.arrow_dataset.Dataset

In [12]:
final_test[21]

{'messages': [{'content': '원고 측의 주장과 피고 측의 주장을 살펴보았을때, 내릴 수 있는 판결을 예측하여 관련 법조항과 결론을 작성해줘.',
   'role': 'system'},
  {'content': '사건유형 민사 세부유형 민사 2심 원고 측의 주장 자동차 도장 공정거래위원회 고시 규정에 따른 물품 제조·수리 관련 계약과 관련하여, 원고는 피고의 부당한 기술자료 유용 행위에 대해 손해를 구하고 있습니다. 요약 피고는 도장 부스에 사용되는 순환수 시스템 악취 제거를 위해 원고에게 미생물제를 제공했으나, 이를 유용하여 신규 미생물을 제작하고 원고와의 거래 관계를 단절했습니다. 이는 공정거래법상 부당한 거래 거절 행위에 해당하며, 원고는 피고에게 10억 원 및 지연손해금을 구하는 명시적 일부청구 소송을 제기하고 있습니다. 피고 측의 주장 피고가 원고에게 물품을 제공하는 데 한정적으로 판단되며, 이는 물품 소비에 해당하여 하도급법적 관계로 보기 어려워 하도급법 위반으로 보기 어려움이 있습니다. 피고가 제공한 물품을 업인으로 판단하여 하도급법적 관계로 보기 어려우며, 관련 자료들은 피고의 업인 차량 등의 제조업에 해당하지 않아 기술자료 유용 행위에 해당하지 않는 것으로 판단됩니다. 기초사실 ## 법률용어 없이 요약 피고(자동차 제조/판매 법인)는 미생물 및 화공약품 악취 문제로 인해 원고(자동차 제조/판매 법인)에게 손해를 입고, 이를 해결하기 위해 도장부스 수처리제(미생물제, 페인트킬러 등)를 공급받고, CWS 공정에서 악취를 저감하는 데 도움을 받았습니다. 피고는 2013년부터 울산 공장 악취 문제 해결을 위해 도장부스 수처리제, VOC 제거제 등을 공급하고, CWS 공정에서 악취를 제거하는 데 도움을 주었습니다. 특히, 2013년 12월 12일에 피고와 O 주식회사가 순환수 집수조 악취저감 시스템 개발 연구 과제를 진행하며 협력 관계를 맺었습니다. 하지만, 피고는 2013년 11월 5일, 2015년까지 특허 등록원부상의 권리를 정리해

#프롬프트 제작 함수

In [13]:
#gradio에서 받은 입력을 바탕으로 프롬프트 제작하는 함수
def make_prompt(court,case,detail_case,question, plaintiff, defendant, fact, conclusion):
    messages = {'messages':
                [
                    {'content': question, 'role': 'system'},
                    {'content': "사건유형 " + case + " 세부유형 " +  detail_case +  " " +  court +
                       " " +  plaintiff + " " + defendant + " " + fact
                       ,'role': 'user'},
                    {'content': conclusion,'role': 'assistant'}
                ]
               }
    return messages

#gradio 설정 및 실행

In [14]:
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            Dropdown1 = gr.Dropdown(["1심", "2심"], label =  "1심, 2심 선택", value="1심", interactive=True)
            Dropdown2 = gr.Dropdown(["민사", "형사", "행정"], label = "사건유형", value="민사", interactive=True)
            Dropdown3 = gr.Dropdown(["민사", "신청", "가사", "특허", "행정", "형사"], label ="세부유형", value="민사", interactive=True)
        with gr.Row():
            text1 = gr.Textbox(label="질문", lines=2)
        with gr.Row():
            text2 = gr.Textbox(label="원고", lines=5, interactive=True)
        with gr.Row():
            text3 = gr.Textbox(label="피고", lines=5, interactive=True)
        with gr.Row():
            text4 = gr.Textbox(label="기초사실", lines=5, interactive=True)
        with gr.Row():
            text5 = gr.Textbox(label="법원 판단", lines=5, interactive=True)

        with gr.Row():
            num = gr.Number(label="인덱스", interactive=True)
            example_btn = gr.Button("예시")
    with gr.Column():
        true_answer = gr.Textbox(label = "정답", lines=5)
        answer = gr.Textbox(label = "예측 결과", lines=5)
        rag = gr.Textbox(label = "참고자료", lines=5)
        answer_btn = gr.Button("변환")


    def example_prompt(n):
        idx = int(n)  # Number는 float로 들어오니까 안전하게 int 변환

        return (
            str(final_test[idx]['messages'][1]['content'][16:18]),
            str(final_test[idx]['messages'][1]['content'][5:7]),
            str(final_test[idx]['messages'][1]['content'][13:15]),
            final_test[idx]['messages'][0]['content'],
            "원고 측의 주장 " + final_test[idx]['messages'][1]['content'].split('피고 측의 주장')[0].split('원고 측의 주장')[1],
            "피고 측의 주장 " + final_test[idx]['messages'][1]['content'].split('피고 측의 주장')[1].split('기초사실')[0],
            "기초사실 " + final_test[idx]['messages'][1]['content'].split('기초사실')[1],
            final_test[idx]['messages'][2]['content'],
        )

    def run_predict(drop1, drop2, drop3, q, plaintiff, defendant, facts, court_judgment):
        """
        버튼 클릭 시 실행되는 함수.
        - 원고/피고/기초사실을 요약한 뒤
        - predict_law 클래스로 추론 실행
        """
        # 빈 문자열 방지용 처리(없어도 상관없긴함)
        plaintiff = plaintiff or ""
        defendant = defendant or ""
        facts = facts or ""
        court_judgment = court_judgment or ""

        # summarize_text 수행 (Summerizer의 summarize_text 사용)
        plaintiff_sum = summarize_text(plaintiff) if plaintiff.strip() else ""
        defendant_sum = summarize_text(defendant) if defendant.strip() else ""
        facts_sum = summarize_text(facts, 400) if facts.strip() else ""

        # predict_law에서 기대하는 데이터 포맷에 맞추기
        data = [
            drop1,           # 0: 1심/2심
            drop2,           # 1: 사건유형
            drop3,           # 2: 세부유형
            q,               # 3: 질문/시스템 프롬프트
            plaintiff_sum,   # 4: 원고 주장 (요약)
            defendant_sum,   # 5: 피고 주장 (요약)
            facts_sum,       # 6: 기초사실 (요약)
            court_judgment,  # 7: 법원 판단 (정답/참고용)
        ]

        predictor = predict_law(data)
        return predictor.prompt['messages'][2]['content'], predictor.answer, predictor.rag  # Gradio의 output으로 전달

    example_btn.click(#버튼 클릭시 test 데이터 샘플 출력
        fn=example_prompt,
        inputs=num,
        outputs=[Dropdown1,Dropdown2,Dropdown3,text1, text2, text3, text4, text5],
    ),
    answer_btn.click(#버튼 클릭시 gradio ui로 입력받은 데이터를 run_predict 함수로 넘겨서 모델 예측 실행
        fn=run_predict,
        inputs=[Dropdown1,Dropdown2,Dropdown3,text1, text2, text3, text4, text5],
        outputs=[true_answer, answer,rag]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6b67d832a77672589.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
